# Solving the p-Laplacian Equation with PhysicsNeMo

This notebook demonstrates how to solve the p-Laplacian partial differential equation using Physics-Informed Neural Networks (PINNs) with the NVIDIA PhysicsNeMo library.

PINNs embed the physics of the problem directly into the neural network training process. The network is trained to minimize a loss function formed from the PDE residual and boundary conditions, effectively solving the differential equation without requiring training data from other solvers. More information about PINNs can be found in the <a href="https://www.sciencedirect.com/science/article/pii/S0021999118307125" rel="nofollow">paper</a> by Raissi et al.

You can refer to the <a href="https://docs.nvidia.com/physicsnemo/index.html" rel="nofollow">PhysicsNeMo User Documentation</a> for more examples and API details.

#### Contents of the Notebook

- [Solving the p-Laplacian PDE](#Solving-the-p-Laplacian-PDE)
    - [Problem Description](#Problem-Description)
    - [Case Setup](#Case-Setup)
    - [Step 1: Creating the geometry](#Step-1:-Creating-the-geometry)
    - [Step 2: Defining the p-Laplacian PDE](#Step-2:-Defining-the-p-Laplacian-PDE)
    - [Step 3: Setting up the Domain and constraints](#Step-3:-Setting-up-the-Domain-and-constraints)
    - [Step 4: Adding Validation data](#Step-4:-Adding-Validation-data)
    - [Step 5: Hydra configuration](#Step-5:-Hydra-configuration)
    - [Step 6: Solver and training](#Step-6:-Solver-and-training)
    - [Visualizing the solution](#Visualizing-the-solution)

#### Learning Outcomes
- How to use PhysicsNeMo to solve PDEs using PINNs
- How to write a custom PDE (the p-Laplacian) in symbolic form
- How to parameterize the PDE (varying p) and train a single network
- How to set up boundary and interior constraints
- How to visualize and compare results across parameter values

## Solving the p-Laplacian PDE

### Problem Description

The **p-Laplacian** equation is a nonlinear generalization of the standard Laplace equation. It arises in many applications including non-Newtonian fluid mechanics, glaciology, and image processing. The equation is:

$$\Delta_p u = \text{div}(|\nabla u|^{p-2} \nabla u) = 0$$

In expanded form for 2D:

$$|\nabla u|^{p-4} \left[(p-1)(u_x^2 u_{xx} + u_y^2 u_{yy}) + 2(p-2) u_x u_y u_{xy} + u_x^2 u_{yy} + u_y^2 u_{xx}\right] = 0$$

For numerical stability, we use the **normalized form** (dividing out the $|\nabla u|^{p-4}$ factor):

$$(p-1) u_x^2 u_{xx} + (p-1) u_y^2 u_{yy} + 2(p-2) u_x u_y u_{xy} + u_x^2 u_{yy} + u_y^2 u_{xx} = 0$$

**Problem Setup:**
- **Domain:** 2D square $[-1, 1] \times [-1, 1]$
- **Boundary condition:** $u(x,y) = |x|^{4/3} - |y|^{4/3}$ on $\partial\Omega$ (Aronsson's function)
- **Parameter:** $p \in [2, 10]$ as a training variable
- **Validation:** Compare solutions at $p = 2, 5, 10$ with the Aronsson solution

The **Aronsson solution** $u(x,y) = |x|^{4/3} - |y|^{4/3}$ is the exact solution for the infinity-Laplacian ($p \to \infty$) and serves as an approximate reference for large $p$.

### Case Setup

Now that we have our problem defined, let's look at the code required to solve it using PhysicsNeMo. The library provides APIs for parameterized geometry (CSG module), symbolic equation definition (SymPy-based), and advanced neural network architectures.

<h4>Note: In this notebook, we describe the contents of the <code>plap.py</code> script</h4>

```python
import numpy as np
from sympy import Symbol, Function, Abs, Rational, sqrt

import physicsnemo.sym
from physicsnemo.sym.hydra import instantiate_arch
from physicsnemo.sym.hydra.config import PhysicsNeMoConfig

from physicsnemo.sym.solver import Solver
from physicsnemo.sym.domain import Domain
from physicsnemo.sym.geometry.primitives_2d import Rectangle
from physicsnemo.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from physicsnemo.sym.domain.validator import PointwiseValidator
from physicsnemo.sym.key import Key
from physicsnemo.sym.eq.pde import PDE
from physicsnemo.sym.utils.io import ValidatorPlotter
```

We import `Rectangle` from the 2D geometry module to define our square domain. The `PDE` base class allows us to define the p-Laplacian symbolically. The `Key` class is used to specify input/output variables for the neural network, and importantly, `p` is included as an input key so a single network learns solutions across all values of $p$.

### Step 1: Creating the geometry

We create a 2D rectangular domain $[-1, 1] \times [-1, 1]$ using the `Rectangle` class. PhysicsNeMo's geometry module also provides circles, triangles, and other shapes for more complex domains.

```python
geo = Rectangle((-1.0, -1.0), (1.0, 1.0))
```

### Step 2: Defining the p-Laplacian PDE

We define the p-Laplacian PDE by inheriting from the `PDE` class. The key idea is to treat $p$ as a **symbolic variable** so that the neural network learns a family of solutions parameterized by $p$.

The normalized p-Laplacian residual is:

$$(p-1) u_x^2 u_{xx} + (p-1) u_y^2 u_{yy} + 2(p-2) u_x u_y u_{xy} + u_x^2 u_{yy} + u_y^2 u_{xx} = 0$$

```python
class PLaplacian(PDE):
    name = "PLaplacian"

    def __init__(self):
        x = Symbol("x")
        y = Symbol("y")
        p = Symbol("p")

        input_variables = {"x": x, "y": y, "p": p}

        u = Function("u")(*input_variables)

        u_x = u.diff(x)
        u_y = u.diff(y)
        u_xx = u.diff(x, x)
        u_yy = u.diff(y, y)
        u_xy = u.diff(x, y)

        p_laplacian = (
            (p - 1) * u_x**2 * u_xx
            + (p - 1) * u_y**2 * u_yy
            + 2 * (p - 2) * u_x * u_y * u_xy
            + u_x**2 * u_yy
            + u_y**2 * u_xx
        )

        self.equations = {}
        self.equations["p_laplacian"] = p_laplacian
```

We defined `x`, `y`, and `p` as SymPy symbols and declared `u` as a function of all three. PhysicsNeMo's symbolic differentiation then computes all the required partial derivatives (`u_x`, `u_y`, `u_xx`, `u_yy`, `u_xy`) automatically. The residual expression is stored in `self.equations["p_laplacian"]`, and setting it to zero in the interior constraint enforces the PDE.

#### Creating the Neural Network nodes

We use a `FullyConnectedArch` with three inputs (`x`, `y`, `p`) and one output (`u`). The network architecture is configured via the Hydra config file. The nodes list combines the PDE nodes (which compute derivatives and the PDE residual) with the network node.

```python
@physicsnemo.sym.main(config_path="conf", config_name="config")
def run(cfg: PhysicsNeMoConfig) -> None:
    p_lap = PLaplacian()

    net = instantiate_arch(
        input_keys=[Key("x"), Key("y"), Key("p")],
        output_keys=[Key("u")],
        cfg=cfg.arch.fully_connected,
    )

    nodes = p_lap.make_nodes() + [net.make_node(name="p_laplacian_network")]
```

### Step 3: Setting up the Domain and constraints

We create a `Domain` and add two types of constraints:

1. **Boundary constraint (BC):** Enforce $u = |x|^{4/3} - |y|^{4/3}$ on the boundary of the square
2. **Interior constraint:** Enforce the p-Laplacian residual $= 0$ inside the domain

Both constraints are parameterized over $p \in [2, 10]$, sampled uniformly during training.

```python
    x_sym = Symbol("x")
    y_sym = Symbol("y")
    p_sym = Symbol("p")
    sdf_sym = Symbol("sdf")

    p_range = {
        p_sym: lambda batch_size: np.random.uniform(2, 10, (batch_size, 1))
    }

    domain = Domain()
```

**Boundary Constraint:** The Aronsson solution $u = |x|^{4/3} - |y|^{4/3}$ is used as the Dirichlet boundary condition. This is expressed symbolically using SymPy's `Abs` and `Rational`.

**Interior Constraint:** The PDE residual `p_laplacian` is set to zero. We apply a weighting scheme:
- **SDF weighting** (`sdf`): Reduces the PDE loss contribution near the boundary where the boundary condition dominates.
- **Axis weighting** ($\sqrt{x^2 + \epsilon^2} \cdot \sqrt{y^2 + \epsilon^2}$): Reduces influence near $x=0$ and $y=0$ where the Aronsson solution has singular gradients ($|x|^{4/3}$ is not smooth at the origin).

```python
    exact_bc = Abs(x_sym)**Rational(4, 3) - Abs(y_sym)**Rational(4, 3)

    BC = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"u": exact_bc},
        lambda_weighting={"u": 1.0},
        batch_size=cfg.batch_size.BC,
        parameterization=p_range,
    )
    domain.add_constraint(BC, "BC")

    eps = 0.01
    dist_x = sqrt(x_sym**2 + eps**2)
    dist_y = sqrt(y_sym**2 + eps**2)
    axis_weight = dist_x * dist_y

    interior = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"p_laplacian": 0},
        batch_size=cfg.batch_size.interior,
        lambda_weighting={
            "p_laplacian": sdf_sym * axis_weight,
        },
        parameterization=p_range,
    )
    domain.add_constraint(interior, "interior")
```

### Step 4: Adding Validation data

We create validation datasets for $p = 2, 5, 10$. For each value of $p$, we evaluate the network on a $50 \times 50$ grid over the domain and compare against the Aronsson solution.

Since the exact solution for finite $p$ is not known analytically, the Aronsson solution (exact for $p \to \infty$) serves as a reference. The comparison becomes more accurate as $p$ increases.

```python
    n_val = 50
    x_vals = np.linspace(-0.98, 0.98, n_val)
    y_vals = np.linspace(-0.98, 0.98, n_val)
    X, Y = np.meshgrid(x_vals, y_vals)
    X = np.expand_dims(X.flatten(), axis=-1)
    Y = np.expand_dims(Y.flatten(), axis=-1)

    u_exact = np.abs(X)**(4/3) - np.abs(Y)**(4/3)

    for p_val in [2, 5, 10]:
        P_test = np.full_like(X, float(p_val))
        invar_p = {"x": X, "y": Y, "p": P_test}
        validator_p = PointwiseValidator(
            nodes=nodes,
            invar=invar_p,
            true_outvar={"u": u_exact},
            batch_size=128,
            plotter=ValidatorPlotter(),
        )
        domain.add_validator(validator_p, f"validation_p{p_val}")
```

### Step 5: Hydra configuration

PhysicsNeMo uses Hydra for configuration management. The config file below defines the optimizer, scheduler, loss function, network architecture, training parameters, and batch sizes. Notable choices:

- **ReLoBRaLo loss:** An adaptive loss balancing scheme that automatically adjusts the relative weighting of different loss terms during training.
- **5-layer, 64-node MLP:** A compact network suitable for this parameterized problem.
- **50,000 training steps** with validation every 2,000 steps.

```yaml
defaults:
  - physicsnemo_default
  - arch:
      - fully_connected
  - scheduler: tf_exponential_lr
  - optimizer: adam
  - loss: relobralo
  - _self_

custom:
  p_min: 2.0
  p_max: 10.0
  p_validation: 10.0

save_filetypes: "vtk,npz"

scheduler:
  decay_rate: 0.95
  decay_steps: 2000

loss:
  alpha: 0.95
  beta: 0.99
  tau: 1.0
  eps: 1.0e-08

training:
  max_steps: 50000
  rec_results_freq: 2000
  rec_validation_freq: 2000
  rec_inference_freq: 2000
  rec_monitor_freq: 200
  rec_constraint_freq: 2000
  save_network_freq: 5000
  print_stats_freq: 200
  summary_freq: 200

batch_size:
  BC: 512
  interior: 2000

arch:
  fully_connected:
    layer_size: 64
    nr_layers: 5
```

### Step 6: Solver and training

We create the solver using the domain and Hydra configuration, then call `solve()` to start training. PhysicsNeMo handles the training loop, logging, checkpointing, and validation automatically.

```python
    slv = Solver(cfg, domain)
    slv.solve()


if __name__ == "__main__":
    run()
```

We are now ready to train the p-Laplacian solver. Since PhysicsNeMo uses Hydra for config management, we run the training as a Python script. The cell below executes `plap.py`.

You can monitor training progress with TensorBoard:

```
tensorboard --logdir outputs/ --port 8889
```

In [ ]:
import os
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost" 

In [ ]:
!python3 plap.py

### Visualizing the solution

PhysicsNeMo saves validation results as `.npz` and `.vtp` files in the `outputs/` directory. Below we load the validation data for each $p$ value and create comparison plots showing the predicted solution, the Aronsson reference, and the error.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import os

plt.rcParams['image.cmap'] = 'jet'

def aronsson_exact(x, y):
    return np.abs(x)**(4/3) - np.abs(y)**(4/3)

# --- Load validation data ---
validator_dir = os.path.join("outputs", "p_laplacian_aronsson", "validators")
p_values = [2, 5, 10]
all_data = {}

for p_val in p_values:
    npz_path = os.path.join(validator_dir, f"validation_p{p_val}.npz")
    raw = np.load(npz_path, allow_pickle=True)
    all_data[p_val] = np.atleast_1d(raw.f.arr_0)[0]
    print(f"Loaded p={p_val}")

# --- Individual validation plots (Predicted / True / Error) ---
for p_val, data in all_data.items():
    x = data['x'].flatten()
    y = data['y'].flatten()
    u_pred = data['pred_u'].flatten()
    u_true = data['true_u'].flatten()

    n_side = int(np.sqrt(len(x)))
    idx = np.lexsort((x, y))
    xg = x[idx].reshape(n_side, n_side)
    yg = y[idx].reshape(n_side, n_side)
    up = u_pred[idx].reshape(n_side, n_side)
    ut = u_true[idx].reshape(n_side, n_side)
    error = up - ut
    rmse = np.sqrt(np.mean(error**2))
    max_err = np.max(np.abs(error))

    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    im0 = axes[0].pcolormesh(xg, yg, up, shading='auto', cmap='jet')
    axes[0].set_title(f'Predicted u (p={p_val})')
    plt.colorbar(im0, ax=axes[0], fraction=0.046)

    im1 = axes[1].pcolormesh(xg, yg, ut, shading='auto', cmap='jet')
    axes[1].set_title('True u (Aronsson)')
    plt.colorbar(im1, ax=axes[1], fraction=0.046)

    im2 = axes[2].pcolormesh(xg, yg, error, shading='auto',
                              cmap='RdBu_r', vmin=-max_err, vmax=max_err)
    axes[2].set_title(f'Error (RMSE={rmse:.4e})')
    plt.colorbar(im2, ax=axes[2], fraction=0.046)

    for ax in axes:
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_aspect('equal')

    plt.suptitle(f'p-Laplacian Validation: p = {p_val}', fontsize=14)
    plt.tight_layout()
    plt.savefig(f"validation_p{p_val}.png", dpi=150, bbox_inches='tight')
    plt.show()

# --- Summary comparison plot (2D contour + 3D surface) ---
n_cols = len(all_data) + 1
fig = plt.figure(figsize=(4 * n_cols, 8))

first_data = list(all_data.values())[0]
x0 = first_data['x'].flatten()
y0 = first_data['y'].flatten()
u_exact_all = aronsson_exact(x0, y0)

n_side = int(np.sqrt(len(x0)))
idx = np.lexsort((x0, y0))
xg = x0[idx].reshape(n_side, n_side)
yg = y0[idx].reshape(n_side, n_side)
u_exact_grid = u_exact_all[idx].reshape(n_side, n_side)

all_u = [d['pred_u'].flatten() for d in all_data.values()]
all_u.append(u_exact_all)
vmin = min(np.min(u) for u in all_u)
vmax = max(np.max(u) for u in all_u)

# Row 1: 2D contour
for i, (p_val, data) in enumerate(sorted(all_data.items())):
    ax = fig.add_subplot(2, n_cols, i + 1)
    u_pred = data['pred_u'].flatten()
    err = u_pred - data['true_u'].flatten()
    rmse = np.sqrt(np.mean(err**2))
    ug = u_pred[idx].reshape(n_side, n_side)
    im = ax.pcolormesh(xg, yg, ug, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
    ax.set_title(f'p={p_val}, RMSE={rmse:.2e}', fontsize=10)
    ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_aspect('equal')
    plt.colorbar(im, ax=ax, fraction=0.046)

ax_exact = fig.add_subplot(2, n_cols, len(all_data) + 1)
im = ax_exact.pcolormesh(xg, yg, u_exact_grid, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
ax_exact.set_title(r'Exact ($\infty$-Lap)', fontsize=10)
ax_exact.set_xlabel('x'); ax_exact.set_ylabel('y'); ax_exact.set_aspect('equal')
plt.colorbar(im, ax=ax_exact, fraction=0.046)

# Row 2: 3D surface
for i, (p_val, data) in enumerate(sorted(all_data.items())):
    ax3d = fig.add_subplot(2, n_cols, n_cols + i + 1, projection='3d')
    ug = data['pred_u'].flatten()[idx].reshape(n_side, n_side)
    ax3d.plot_surface(xg, yg, ug, cmap='jet', vmin=vmin, vmax=vmax,
                      linewidth=0, antialiased=True)
    ax3d.set_xlabel('x'); ax3d.set_ylabel('y'); ax3d.set_zlabel('u')
    ax3d.set_title(f'p={p_val}', fontsize=10)
    ax3d.view_init(elev=25, azim=-60)

ax3d_exact = fig.add_subplot(2, n_cols, 2 * n_cols, projection='3d')
ax3d_exact.plot_surface(xg, yg, u_exact_grid, cmap='jet', vmin=vmin, vmax=vmax,
                        linewidth=0, antialiased=True)
ax3d_exact.set_xlabel('x'); ax3d_exact.set_ylabel('y'); ax3d_exact.set_zlabel('u')
ax3d_exact.set_title(r'Exact ($\infty$-Lap)', fontsize=10)
ax3d_exact.view_init(elev=25, azim=-60)

plt.suptitle('p-Laplacian: Comparison Across p Values', fontsize=14)
plt.tight_layout()
plt.savefig("summary_comparison.png", dpi=150, bbox_inches='tight')
plt.show()

print("Done plotting.")

We have now trained and visualized the p-Laplacian equation using PINNs with PhysicsNeMo. The network learns a **family of solutions** parameterized by $p$, allowing us to evaluate the solution at any value of $p$ within the training range without retraining.

As $p$ increases, the solution should converge toward the Aronsson solution $u(x,y) = |x|^{4/3} - |y|^{4/3}$, which is the exact infinity-Laplacian solution.